In [1]:
from email_extractor import get_emails_df # local package
import pandas as pd

In [2]:
SPAM_1_PATH = 'spam-data/spam_assassin_corpus/spam_1'
SPAM_2_PATH = 'spam-data/spam_assassin_corpus/spam_2'
EASY_NONSPAM_1_PATH = 'spam-data/spam_assassin_corpus/easy_ham_1'
EASY_NONSPAM_2_PATH = 'spam-data/spam_assassin_corpus/easy_ham_2'

SPAM_CATEGORY = 1
HAM_CATEGORY = 0

# *Gather data:*
### *load spam and ham body email DataFrames*

In [3]:
spam_emails = get_emails_df(SPAM_1_PATH, SPAM_CATEGORY)
spam_emails = spam_emails.append(get_emails_df(SPAM_2_PATH, SPAM_CATEGORY))

print(spam_emails.shape)
spam_emails.head()

(1898, 2)


,MESSAGE,CATEGORY
00279.1d58a13e343c1e53aca2ed2121a3f815,"5, Meridian East\n\nLeicester LE3 2WZ \n\nLeic...",1
00380.a262abe251ca7cc3026e4e146d9cf817,"<html>\n\n<TABLE id=AutoNumber2 style=""BORDER-...",1
00176.79f82496c612ea28f45f13ca5c47f8c2,<html>\n\n<head>\n\n<title></title>\n\n</head>...,1
00461.1a27d007492d1c665d07db820b7dc3b8,<html>\n\n<head>\n\n<title>Tell Me Where to Se...,1
00300.834f370a21ca4f1774d5724b5443411c,"<HTML>\n\n<BODY>\n\n<meta http-equiv=""Content-...",1


In [4]:
ham_emails = get_emails_df(EASY_NONSPAM_1_PATH, HAM_CATEGORY)
ham_emails = ham_emails.append(get_emails_df(EASY_NONSPAM_2_PATH, HAM_CATEGORY))

print(ham_emails.shape)
ham_emails.head()

(3901, 2)


,MESSAGE,CATEGORY
02152.8df514c41920019281f8f0723dad0001,URL: http://www.aaronsw.com/weblog/000622\n\nD...,0
01482.a3576ddec34b0481a9a671c1f3c141c3,"Tony L. Svanstrom wrote:\n\n\n\n>On Sun, 15 Se...",0
00152.703c271de3d42fb8bf266db6f77a0dda,"Thanks for the info AJ, I found ""weblog"" at \n...",0
00699.29e599983f044aee500f3a58c34acffc,"Gary Lawrence Murphy wrote:\n\n\n\n>>>>>>""R"" =...",0
01846.af884c94693e5cce571396b4fd151167,URL: http://boingboing.net/#85481660\n\nDate: ...,0


#### *full-data (spam/nonspam)*

In [5]:
data = pd.concat([spam_emails, ham_emails])
data.shape

(5799, 2)